<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/Tensor_Network_Expansions_(Step).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorly

In [ ]:
import numpy as np
import opt_einsum as oe

class PEPS:
    def __init__(self, Lx, Ly, d, D):
        """
        Initialize a PEPS tensor network.
        Args:
            Lx (int): Number of sites in the x-direction (rows).
            Ly (int): Number of sites in the y-direction (columns).
            d (int): Physical dimension of the local Hilbert space.
            D (int): Bond dimension of the virtual indices.
        """
        self.Lx = Lx
        self.Ly = Ly
        self.d = d
        self.D = D

        # Initialize tensors for each site in the 2D lattice
        self.tensors = [[self.initialize_tensor(d, D) for _ in range(Ly)] for _ in range(Lx)]

    def initialize_tensor(self, d, D):
        """
        Initialize a random PEPS tensor with physical and virtual bonds.
        """
        return np.random.randn(d, D, D, D, D)  # Physical, Left, Right, Up, Down

    def contract_bond(self, tensor_a, tensor_b, axis_a, axis_b):
        """
        Contract two tensors along specified axes.
        Args:
            tensor_a: First tensor.
            tensor_b: Second tensor.
            axis_a: Axis to contract in tensor_a.
            axis_b: Axis to contract in tensor_b.
        Returns:
            Contracted tensor.
        """
        # Debug tensor shapes
        print(f"Contracting tensors: {tensor_a.shape} with {tensor_b.shape}")

        # Ensure axis indices are positive and within bounds
        axis_a = axis_a if axis_a >= 0 else tensor_a.ndim + axis_a
        axis_b = axis_b if axis_b >= 0 else tensor_b.ndim + axis_b

        # Validate shapes
        if tensor_a.shape[axis_a] != tensor_b.shape[axis_b]:
            raise ValueError(
                f"Bond dimensions do not match: {tensor_a.shape[axis_a]} != {tensor_b.shape[axis_b]}"
            )

        # Perform contraction
        return oe.contract(
            tensor_a,
            list(range(tensor_a.ndim)),
            tensor_b,
            list(range(tensor_b.ndim)),
            optimize="optimal"
        )

    def contract_row(self, row_tensors):
        """
        Contract all tensors in a single row to form a single tensor.
        Args:
            row_tensors: List of tensors in a row.
        Returns:
            Contracted row tensor.
        """
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            # If the tensor is already a scalar, stop contracting
            if contracted.ndim == 0:
                break
            contracted = self.contract_bond(contracted, t, axis_a=-1, axis_b=1)
        return contracted

    def full_contraction(self):
        """
        Perform a full contraction of the PEPS lattice.
        Returns:
            Scalar value of the fully contracted network.
        """
        # Contract rows sequentially
        row_contractions = [self.contract_row(row) for row in self.tensors]
        result = row_contractions[0]
        for row in row_contractions[1:]:
            # If the tensor is already a scalar, stop contracting
            if result.ndim == 0:
                break
            result = self.contract_bond(result, row, axis_a=-1, axis_b=1)
        return result

# Example Usage
Lx, Ly = 4, 4  # Lattice dimensions
d, D = 2, 3    # Physical and bond dimensions

peps = PEPS(Lx, Ly, d, D)
result = peps.full_contraction()

print(f"Full contraction result: {result}")

In [ ]:
import numpy as np
import opt_einsum as oe
import torch  # For parallelized GPU computations


class PEPS:
    def __init__(self, Lx, Ly, d, D, use_gpu=False):
        """
        Initialize a PEPS tensor network.
        Args:
            Lx (int): Number of sites in the x-direction (rows).
            Ly (int): Number of sites in the y-direction (columns).
            d (int): Physical dimension of the local Hilbert space.
            D (int): Bond dimension of the virtual indices.
            use_gpu (bool): Enable GPU acceleration using PyTorch.
        """
        self.Lx = Lx
        self.Ly = Ly
        self.d = d
        self.D = D
        self.use_gpu = use_gpu

        # Initialize tensors for each site in the 2D lattice
        self.tensors = [[self.initialize_tensor(d, D) for _ in range(Ly)] for _ in range(Lx)]

    def initialize_tensor(self, d, D):
        """
        Initialize a random PEPS tensor with physical and virtual bonds.
        """
        tensor = np.random.randn(d, D, D, D, D)  # Physical, Left, Right, Up, Down
        if self.use_gpu:
            tensor = torch.tensor(tensor, dtype=torch.float32).to('cuda')
        return tensor

    def contract_bond(self, tensor_a, tensor_b, axis_a, axis_b):
        """
        Contract two tensors along specified axes.
        Args:
            tensor_a: First tensor.
            tensor_b: Second tensor.
            axis_a: Axis to contract in tensor_a.
            axis_b: Axis to contract in tensor_b.
        Returns:
            Contracted tensor.
        """
        axis_a = axis_a if axis_a >= 0 else tensor_a.ndim + axis_a
        axis_b = axis_b if axis_b >= 0 else tensor_b.ndim + axis_b
        if tensor_a.shape[axis_a] != tensor_b.shape[axis_b]:
            raise ValueError(
                f"Bond dimensions do not match: {tensor_a.shape[axis_a]} != {tensor_b.shape[axis_b]}"
            )
        return oe.contract(
            tensor_a,
            list(range(tensor_a.ndim)),
            tensor_b,
            list(range(tensor_b.ndim)),
            optimize="optimal"
        )

    def contract_row(self, row_tensors):
        """
        Contract all tensors in a single row to form a single tensor.
        Args:
            row_tensors: List of tensors in a row.
        Returns:
            Contracted row tensor.
        """
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            if contracted.ndim == 0:  # Stop if tensor is a scalar
                break
            contracted = self.contract_bond(contracted, t, axis_a=-1, axis_b=1)
        return contracted

    def full_contraction(self):
        """
        Perform a full contraction of the PEPS lattice.
        Returns:
            Scalar value of the fully contracted network.
        """
        row_contractions = [self.contract_row(row) for row in self.tensors]
        result = row_contractions[0]
        for row in row_contractions[1:]:
            if result.ndim == 0:  # Stop if tensor is a scalar
                break
            result = self.contract_bond(result, row, axis_a=-1, axis_b=1)
        return result

    def apply_hamiltonian(self, h):
        """
        Apply a Hamiltonian to the PEPS lattice.
        Args:
            h: Hamiltonian tensor (physical dimension x physical dimension).
        """
        print(f"Applying Hamiltonian: {h.shape}")
        for i in range(self.Lx):
            for j in range(self.Ly):
                tensor = self.tensors[i][j]
                self.tensors[i][j] = oe.contract("ab,bijkl->aijkl", h, tensor)

    def compute_observable(self, observable):
        """
        Compute an observable using the PEPS framework.
        Args:
            observable: Observable tensor (physical dimension x physical dimension).
        Returns:
            Scalar value of the observable.
        """
        print(f"Computing observable: {observable.shape}")
        result = 0.0
        for row in self.tensors:
            for tensor in row:
                # Apply observable only on the physical index
                physical_contracted = oe.contract("ab,aijkl->bijkl", observable, tensor)
                result += physical_contracted.sum()
        return result

    def corner_transfer_matrix(self):
        """
        Implement a simplified version of CTM for boundary approximations.
        This is a placeholder for further development.
        """
        print("CTM approximation placeholder - not yet implemented.")
        return None


# Example Usage
Lx, Ly = 4, 4  # Lattice dimensions
d, D = 2, 3    # Physical and bond dimensions

# Enable GPU if available
use_gpu = torch.cuda.is_available()

peps = PEPS(Lx, Ly, d, D, use_gpu=use_gpu)

# Example Hamiltonian
hamiltonian = np.random.randn(d, d)
if use_gpu:
    hamiltonian = torch.tensor(hamiltonian, dtype=torch.float32).to('cuda')

peps.apply_hamiltonian(hamiltonian)

# Compute an observable
observable = np.random.randn(d, d)
if use_gpu:
    observable = torch.tensor(observable, dtype=torch.float32).to('cuda')

observable_result = peps.compute_observable(observable)
print(f"Observable result: {observable_result}")

# Perform full contraction
result = peps.full_contraction()
print(f"Full contraction result: {result}")

In [ ]:
import numpy as np
import opt_einsum as oe
import torch


class PEPS:
    def __init__(self, Lx, Ly, d, D, use_gpu=False):
        self.Lx = Lx
        self.Ly = Ly
        self.d = d
        self.D = D
        self.use_gpu = use_gpu
        self.tensors = [[self.initialize_tensor(d, D) for _ in range(Ly)] for _ in range(Lx)]

    def initialize_tensor(self, d, D):
        tensor = np.random.randn(d, D, D, D, D)  # Physical, Left, Right, Up, Down
        if self.use_gpu:
            tensor = torch.tensor(tensor, dtype=torch.float32).to('cuda')
        return tensor

    def contract_bond(self, tensor_a, tensor_b, axis_a, axis_b):
        axis_a = axis_a if axis_a >= 0 else tensor_a.ndim + axis_a
        axis_b = axis_b if axis_b >= 0 else tensor_b.ndim + axis_b
        if tensor_a.shape[axis_a] != tensor_b.shape[axis_b]:
            raise ValueError(
                f"Bond dimensions do not match: {tensor_a.shape[axis_a]} != {tensor_b.shape[axis_b]}"
            )
        return oe.contract(
            tensor_a,
            list(range(tensor_a.ndim)),
            tensor_b,
            list(range(tensor_b.ndim)),
            optimize="optimal"
        )

    def contract_row(self, row_tensors):
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            if contracted.ndim == 0:
                break
            contracted = self.contract_bond(contracted, t, axis_a=-1, axis_b=1)
        return contracted

    def full_contraction(self):
        row_contractions = [self.contract_row(row) for row in self.tensors]
        result = row_contractions[0]
        for row in row_contractions[1:]:
            if result.ndim == 0:
                break
            result = self.contract_bond(result, row, axis_a=-1, axis_b=1)
        return result

    def apply_hamiltonian(self, h):
        print(f"Applying Hamiltonian: {h.shape}")
        for i in range(self.Lx):
            for j in range(self.Ly):
                tensor = self.tensors[i][j]
                self.tensors[i][j] = oe.contract("ab,bijkl->aijkl", h, tensor)

    def compute_observable(self, observable):
        print(f"Computing observable: {observable.shape}")
        result = 0.0
        for row in self.tensors:
            for tensor in row:
                physical_contracted = oe.contract("ab,aijkl->bijkl", observable, tensor)
                result += physical_contracted.sum()
        return result

    def compute_multi_site_observable(self, observable, sites):
        print(f"Computing multi-site observable on sites: {sites}")
        contracted_tensors = []
        for (i, j) in sites:
            tensor = self.tensors[i][j]
            # Align physical dimension to observable if mismatched
            if tensor.shape[0] != observable.shape[0]:
                raise ValueError(f"Physical dimension mismatch: {tensor.shape[0]} != {observable.shape[0]}")
            physical_contracted = oe.contract("ab,aijkl->bijkl", observable, tensor)
            contracted_tensors.append(physical_contracted)

        # Sequentially contract the tensors
        result = contracted_tensors[0]
        for t in contracted_tensors[1:]:
            # Align virtual dimensions before contraction
            if result.shape[-1] != t.shape[1]:
                raise ValueError(f"Bond mismatch during multi-site contraction: {result.shape[-1]} != {t.shape[1]}")
            result = self.contract_bond(result, t, axis_a=-1, axis_b=1)
        return result.sum()

    def corner_transfer_matrix(self, steps=2):
        print(f"Running CTM approximation for {steps} steps...")
        for step in range(steps):
            for i in range(self.Lx):
                for j in range(self.Ly):
                    tensor = self.tensors[i][j]
                    self.tensors[i][j] = tensor / tensor.norm() if self.use_gpu else tensor / np.linalg.norm(tensor)
        print("CTM approximation completed.")

    def optimize_tensors(self):
        print("Optimizing tensors using GPU... (placeholder)")


# Example Usage
Lx, Ly = 4, 4  # Lattice dimensions
d, D = 2, 3    # Physical and bond dimensions

# Enable GPU if available
use_gpu = torch.cuda.is_available()

peps = PEPS(Lx, Ly, d, D, use_gpu=use_gpu)

# Example Hamiltonian
hamiltonian = np.random.randn(d, d)
if use_gpu:
    hamiltonian = torch.tensor(hamiltonian, dtype=torch.float32).to('cuda')

peps.apply_hamiltonian(hamiltonian)

# Compute an observable
observable = np.random.randn(d, d)
if use_gpu:
    observable = torch.tensor(observable, dtype=torch.float32).to('cuda')

observable_result = peps.compute_observable(observable)
print(f"Observable result: {observable_result}")

# Compute a multi-site observable
multi_site_observable = np.random.randn(d, d)
if use_gpu:
    multi_site_observable = torch.tensor(multi_site_observable, dtype=torch.float32).to('cuda')

# Ensure physical dimension alignment
if multi_site_observable.shape[0] != d:
    raise ValueError("Physical dimension of the observable does not match PEPS tensors.")

multi_site_result = peps.compute_multi_site_observable(multi_site_observable, [(0, 0), (1, 1)])
print(f"Multi-site observable result: {multi_site_result}")

# Perform full contraction
result = peps.full_contraction()
print(f"Full contraction result: {result}")

# Perform boundary contraction via CTM
peps.corner_transfer_matrix(steps=5)

In [ ]:
import numpy as np
import opt_einsum as oe
import torch

class PEPS:
    def __init__(self, Lx, Ly, d, D, use_gpu=False):
        self.Lx = Lx
        self.Ly = Ly
        self.d = d
        self.D = D
        self.use_gpu = use_gpu
        self.tensors = [[self.initialize_tensor(d, D) for _ in range(Ly)] for _ in range(Lx)]

    def initialize_tensor(self, d, D):
        tensor = np.random.randn(d, D, D, D, D)  # Physical, Left, Right, Up, Down
        if self.use_gpu:
            tensor = torch.tensor(tensor, dtype=torch.float32).to('cuda')
        return tensor

    def contract_bond(self, tensor_a, tensor_b, axis_a, axis_b):
        axis_a = axis_a if axis_a >= 0 else tensor_a.ndim + axis_a
        axis_b = axis_b if axis_b >= 0 else tensor_b.ndim + axis_b
        if tensor_a.shape[axis_a] != tensor_b.shape[axis_b]:
            raise ValueError(
                f"Bond dimensions do not match: {tensor_a.shape[axis_a]} != {tensor_b.shape[axis_b]}"
            )
        return oe.contract(
            tensor_a,
            list(range(tensor_a.ndim)),
            tensor_b,
            list(range(tensor_b.ndim)),
            optimize="optimal"
        )

    def contract_row(self, row_tensors):
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            if contracted.ndim == 0:
                break
            contracted = self.contract_bond(contracted, t, axis_a=-1, axis_b=1)
        return contracted

    def full_contraction(self):
        row_contractions = [self.contract_row(row) for row in self.tensors]
        result = row_contractions[0]
        for row in row_contractions[1:]:
            if result.ndim == 0:
                break
            result = self.contract_bond(result, row, axis_a=-1, axis_b=1)
        return result

    def apply_hamiltonian(self, h):
        print(f"Applying Hamiltonian: {h.shape}")
        for i in range(self.Lx):
            for j in range(self.Ly):
                tensor = self.tensors[i][j]
                self.tensors[i][j] = oe.contract("ab,bijkl->aijkl", h, tensor)

    def compute_observable(self, observable):
        print(f"Computing observable: {observable.shape}")
        result = 0.0
        for row in self.tensors:
            for tensor in row:
                physical_contracted = oe.contract("ab,aijkl->bijkl", observable, tensor)
                result += physical_contracted.sum()
        return result

    def compute_multi_site_observable(self, observable, sites):
        print(f"Computing multi-site observable on sites: {sites}")
        contracted_tensors = []
        for (i, j) in sites:
            tensor = self.tensors[i][j]
            if tensor.shape[0] != observable.shape[0]:
                raise ValueError(f"Physical dimension mismatch: {tensor.shape[0]} != {observable.shape[0]}")
            physical_contracted = oe.contract("ab,aijkl->bijkl", observable, tensor)
            contracted_tensors.append(physical_contracted)

        result = contracted_tensors[0]
        for t in contracted_tensors[1:]:
            if result.shape[-1] != t.shape[1]:
                raise ValueError(f"Bond mismatch during multi-site contraction: {result.shape[-1]} != {t.shape[1]}")
            result = self.contract_bond(result, t, axis_a=-1, axis_b=1)
        return result.sum()

    def corner_transfer_matrix(self, steps=2):
        print(f"Running CTM approximation for {steps} steps...")
        for step in range(steps):
            for i in range(self.Lx):
                for j in range(self.Ly):
                    tensor = self.tensors[i][j]
                    self.tensors[i][j] = tensor / tensor.norm() if self.use_gpu else tensor / np.linalg.norm(tensor)
        print("CTM approximation completed.")

    def optimize_tensors(self):
        print("Optimizing tensors using GPU... (placeholder)")


# Example Usage
Lx, Ly = 4, 4
d, D = 2, 3

use_gpu = torch.cuda.is_available()

peps = PEPS(Lx, Ly, d, D, use_gpu=use_gpu)

hamiltonian = np.random.randn(d, d)
if use_gpu:
    hamiltonian = torch.tensor(hamiltonian, dtype=torch.float32).to('cuda')

peps.apply_hamiltonian(hamiltonian)

observable = np.random.randn(d, d)
if use_gpu:
    observable = torch.tensor(observable, dtype=torch.float32).to('cuda')

observable_result = peps.compute_observable(observable)
print(f"Observable result: {observable_result}")

multi_site_observable = np.random.randn(d, d)
if use_gpu:
    multi_site_observable = torch.tensor(multi_site_observable, dtype=torch.float32).to('cuda')

if multi_site_observable.shape[0] != d:
    raise ValueError("Physical dimension of the observable does not match PEPS tensors.")

multi_site_result = peps.compute_multi_site_observable(multi_site_observable, [(0, 0), (1, 1)])
print(f"Multi-site observable result: {multi_site_result}")

result = peps.full_contraction()
print(f"Full contraction result: {result}")

peps.corner_transfer_matrix(steps=5)

In [ ]:
import numpy as np
import opt_einsum as oe
import torch


class PEPS:
    def __init__(self, Lx, Ly, d, D, use_gpu=False):
        self.Lx = Lx  # Number of rows
        self.Ly = Ly  # Number of columns
        self.d = d    # Physical dimension
        self.D = D    # Bond dimension
        self.use_gpu = use_gpu

        # Initialize tensors
        self.tensors = [
            [
                self.initialize_tensor(d, D)
                for _ in range(Ly)
            ]
            for _ in range(Lx)
        ]

    def initialize_tensor(self, d, D):
        """
        Initialize a PEPS tensor with shape (physical, left, right, up, down).
        """
        tensor = np.random.randn(d, D, D, D, D)
        if self.use_gpu:
            tensor = torch.tensor(tensor, dtype=torch.float32).to('cuda')
        else:
            tensor = torch.tensor(tensor, dtype=torch.float32)
        return tensor

    def apply_hamiltonian(self, h):
        """
        Apply a local Hamiltonian to each tensor.
        """
        print(f"Applying Hamiltonian: {h.shape}")
        for i in range(self.Lx):
            for j in range(self.Ly):
                tensor = self.tensors[i][j]
                self.tensors[i][j] = oe.contract("ab,bijkl->aijkl", h, tensor)

    def compute_observable(self, observable):
        """
        Compute an observable over the entire PEPS.
        """
        print(f"Computing observable: {observable.shape}")
        result = 0.0
        for row in self.tensors:
            for tensor in row:
                physical_contracted = oe.contract("ab,aijkl->bijkl", observable, tensor)
                result += physical_contracted.sum().item()
        return result

    def compute_multi_site_observable(self, observable, sites):
        """
        Compute a multi-site observable over specified sites.
        """
        print(f"Computing multi-site observable on sites: {sites}")
        contracted_tensors = []
        for (i, j) in sites:
            tensor = self.tensors[i][j]
            physical_contracted = oe.contract("ab,aijkl->bijkl", observable, tensor)
            contracted_tensors.append(physical_contracted)

        result = contracted_tensors[0]
        for idx, t in enumerate(contracted_tensors[1:]):
            print(f"Contracting tensor {idx+1}: result.shape={result.shape}, t.shape={t.shape}")
            result, t = self.ensure_bond_match(result, t)
            result = oe.contract("...ij,...jk->...ik", result, t)
        return result.sum().item()

    def full_contraction(self):
        """
        Perform a full contraction of the PEPS.
        """
        print("Performing full contraction...")
        row_contractions = []
        for row_idx, row in enumerate(self.tensors):
            print(f"Contracting row {row_idx}...")
            row_contracted = self.contract_row(row)
            print(f"Row {row_idx} contraction result: shape={row_contracted.shape}")
            row_contractions.append(row_contracted)

        result = row_contractions[0]
        for i in range(1, len(row_contractions)):
            print(
                f"Contracting rows: result.shape={result.shape}, "
                f"row_contractions[{i}].shape={row_contractions[i].shape}"
            )
            result, row_contractions[i] = self.ensure_bond_match(result, row_contractions[i])
            result = oe.contract("...ij,...jk->...ik", result, row_contractions[i])
        return result.sum().item()

    def contract_row(self, row_tensors):
        """
        Contract tensors in a single row along the horizontal bonds.
        """
        contracted = row_tensors[0]
        for idx, t in enumerate(row_tensors[1:]):
            print(
                f"Contracting row tensors: contracted.shape={contracted.shape}, "
                f"t[{idx+1}].shape={t.shape}"
            )
            contracted, t = self.ensure_bond_match(contracted, t)
            contracted = oe.contract("...lm,...mn->...ln", contracted, t)
        return contracted

    def ensure_bond_match(self, t1, t2):
        """
        Ensures that bond dimensions of two tensors are aligned via zero-padding.
        """
        def pad_tensor(t, target_shape):
            """
            Zero-pad a tensor to the target shape.
            """
            pad_sizes = [(0, max(0, target_dim - current_dim)) for current_dim, target_dim in zip(t.shape, target_shape)]
            pad_sizes = [item for sublist in reversed(pad_sizes) for item in sublist]
            return torch.nn.functional.pad(t, pad_sizes)

        # Print tensor shapes for debugging
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")

        # Pad all dimensions to the maximum shape
        max_shape = tuple(max(s1, s2) for s1, s2 in zip(t1.shape, t2.shape))
        t1_padded = pad_tensor(t1, max_shape)
        t2_padded = pad_tensor(t2, max_shape)

        print(f"After padding: t1_padded.shape={t1_padded.shape}, t2_padded.shape={t2_padded.shape}")
        return t1_padded, t2_padded


# Example Usage
if __name__ == "__main__":
    # Lattice dimensions
    Lx, Ly = 4, 4
    # Physical and bond dimensions
    d, D = 2, 3
    # Check for GPU
    use_gpu = torch.cuda.is_available()

    # Initialize PEPS
    peps = PEPS(Lx, Ly, d, D, use_gpu=use_gpu)

    # Initialize random Hamiltonian
    hamiltonian = np.random.randn(d, d)
    if use_gpu:
        hamiltonian = torch.tensor(hamiltonian, dtype=torch.float32).to('cuda')
    else:
        hamiltonian = torch.tensor(hamiltonian, dtype=torch.float32)

    # Apply Hamiltonian
    peps.apply_hamiltonian(hamiltonian)

    # Initialize random observable
    observable = np.random.randn(d, d)
    if use_gpu:
        observable = torch.tensor(observable, dtype=torch.float32).to('cuda')
    else:
        observable = torch.tensor(observable, dtype=torch.float32)

    # Compute single-site observable
    observable_result = peps.compute_observable(observable)
    print(f"Observable result: {observable_result}")

    # Compute multi-site observable
    multi_site_observable = np.random.randn(d, d)
    if use_gpu:
        multi_site_observable = torch.tensor(multi_site_observable, dtype=torch.float32).to('cuda')
    else:
        multi_site_observable = torch.tensor(multi_site_observable, dtype=torch.float32)

    multi_site_result = peps.compute_multi_site_observable(multi_site_observable, [(0, 0), (1, 1)])
    print(f"Multi-site observable result: {multi_site_result}")

    # Perform full contraction
    result = peps.full_contraction()
    print(f"Full contraction result: {result}")

In [ ]:
ls | grep torch

In [ ]:
!pip uninstall torch -y
!pip install torch --upgrade

In [ ]:
import torch
print(torch.__version__)

In [ ]:
!python -m venv clean_env
!source clean_env/bin/activate  # Use clean_env\Scripts\activate on Windows
!pip install torch opt_einsum

In [ ]:
import torch
import opt_einsum as oe

class PEPS:
    def __init__(self, size, bond_dim):
        self.size = size
        self.bond_dim = bond_dim
        self.tensors = self.initialize_tensors()

    def initialize_tensors(self):
        tensors = []
        for i in range(self.size[0]):
            row = []
            for j in range(self.size[1]):
                tensor_shape = [self.bond_dim] * 4
                row.append(torch.rand(*tensor_shape, requires_grad=True))
            tensors.append(row)
        return tensors

    def apply_hamiltonian(self):
        h = torch.rand(self.size, requires_grad=True)
        print(f"Applying Hamiltonian: {h.shape}")
        return h

    def compute_observable(self):
        obs = torch.rand(self.size, requires_grad=True)
        print(f"Computing observable: {obs.shape}")
        return obs

    def ensure_bond_match(self, t1, t2):
        """
        Ensure that the bond dimensions between t1 and t2 match.
        """
        if t1.shape[-1] != t2.shape[0]:
            raise ValueError("Bond dimensions mismatch. Tensor contraction invalid.")
        return t1, t2

    def contract_row(self, row_tensors):
        """
        Contract a row of tensors into a single tensor.
        """
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            contracted, t = self.ensure_bond_match(contracted, t)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, t)
        return contracted

    def full_contraction(self):
        """
        Perform a full contraction of the PEPS tensor network.
        """
        row_contractions = []
        for row in self.tensors:
            row_contracted = self.contract_row(row)
            row_contractions.append(row_contracted)

        contracted = row_contractions[0]
        for row in row_contractions[1:]:
            contracted, row = self.ensure_bond_match(contracted, row)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, row)

        return contracted

# Define the PEPS model
size = (3, 3)
bond_dim = 3
peps = PEPS(size, bond_dim)

# Apply Hamiltonian
hamiltonian = peps.apply_hamiltonian()

# Compute observable
observable = peps.compute_observable()

# Print observable result
observable_result = observable.sum()
print(f"Observable result: {observable_result.item()}")

# Perform full contraction
full_contraction_result = peps.full_contraction()
print(f"Full contraction result: {full_contraction_result}")

In [ ]:
import torch
import opt_einsum as oe

class PEPS:
    def __init__(self, size, bond_dim):
        self.size = size  # PEPS lattice dimensions (rows, columns)
        self.bond_dim = bond_dim  # Bond dimension of the PEPS
        self.tensors = self.initialize_tensors()

    def initialize_tensors(self):
        """
        Initialize the PEPS tensors with random values.
        Each tensor is of shape [bond_dim, bond_dim, bond_dim, bond_dim].
        """
        tensors = []
        for i in range(self.size[0]):
            row = []
            for j in range(self.size[1]):
                tensor_shape = [self.bond_dim] * 4
                row.append(torch.rand(*tensor_shape, requires_grad=True))
            tensors.append(row)
        return tensors

    def apply_hamiltonian(self):
        """
        Apply a Hamiltonian to the PEPS lattice. For simplicity, this is a random tensor.
        """
        h = torch.rand(self.size, requires_grad=True)
        print(f"Applying Hamiltonian: {h.shape}")
        return h

    def compute_observable(self):
        """
        Compute a random observable on the PEPS lattice for demonstration purposes.
        """
        obs = torch.rand(self.size, requires_grad=True)
        print(f"Computing observable: {obs.shape}")
        return obs

    def ensure_bond_match(self, t1, t2):
        """
        Ensure that the bond dimensions between t1 and t2 match.
        """
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")
        if t1.shape[-1] != t2.shape[0]:
            raise ValueError("Bond dimensions mismatch. Tensor contraction invalid.")
        return t1, t2

    def contract_row(self, row_tensors):
        """
        Contract a row of tensors into a single tensor.
        """
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            contracted, t = self.ensure_bond_match(contracted, t)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, t)
        return contracted

    def full_contraction(self):
        """
        Perform a full contraction of the PEPS tensor network.
        """
        row_contractions = []
        for row in self.tensors:
            row_contracted = self.contract_row(row)
            row_contractions.append(row_contracted)

        # Contract all rows together
        contracted = row_contractions[0]
        for row in row_contractions[1:]:
            contracted, row = self.ensure_bond_match(contracted, row)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, row)

        return contracted

# Define the PEPS model
size = (3, 3)  # PEPS lattice size (3x3 grid)
bond_dim = 3  # Bond dimension
peps = PEPS(size, bond_dim)

# Apply Hamiltonian
hamiltonian = peps.apply_hamiltonian()

# Compute observable
observable = peps.compute_observable()

# Print observable result
observable_result = observable.sum()
print(f"Observable result: {observable_result.item()}")

# Perform full contraction and reduce to scalar
full_contraction_result = peps.full_contraction()
final_result = full_contraction_result.sum()
print(f"Full contraction result (scalar): {final_result.item()}")

In [ ]:
import torch
import opt_einsum as oe

class PEPS:
    def __init__(self, size, bond_dim):
        """
        Initialize a PEPS model with a given lattice size and bond dimension.

        Parameters:
        size: Tuple[int, int] - The PEPS lattice dimensions (rows, columns).
        bond_dim: int - Bond dimension for the PEPS tensors.
        """
        self.size = size
        self.bond_dim = bond_dim
        self.tensors = self.initialize_tensors()

    def initialize_tensors(self):
        """
        Randomly initialize the PEPS tensors for each lattice site.

        Returns:
        List[List[Tensor]]: A 2D grid of tensors, each of shape (bond_dim, bond_dim, bond_dim, bond_dim).
        """
        tensors = []
        for i in range(self.size[0]):
            row = []
            for j in range(self.size[1]):
                tensor_shape = [self.bond_dim] * 4
                row.append(torch.rand(*tensor_shape, requires_grad=True))
            tensors.append(row)
        return tensors

    def apply_hamiltonian(self):
        """
        Apply a Hamiltonian to the PEPS lattice (dummy implementation with random tensor).

        Returns:
        Tensor: A tensor representing the Hamiltonian (random for demonstration).
        """
        h = torch.rand(self.size, requires_grad=True)
        print(f"Applying Hamiltonian: {h.shape}")
        return h

    def compute_observable(self):
        """
        Compute a dummy observable on the PEPS lattice (random tensor for demonstration).

        Returns:
        Tensor: A tensor representing the observable (random for demonstration).
        """
        obs = torch.rand(self.size, requires_grad=True)
        print(f"Computing observable: {obs.shape}")
        return obs

    def ensure_bond_match(self, t1, t2):
        """
        Check that the bond dimensions of two tensors match for contraction.

        Parameters:
        t1, t2: Tensor - Tensors to be contracted.

        Returns:
        Tuple[Tensor, Tensor]: The original tensors after validation.
        """
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")
        if t1.shape[-1] != t2.shape[0]:
            raise ValueError("Bond dimensions mismatch. Tensor contraction invalid.")
        return t1, t2

    def contract_row(self, row_tensors):
        """
        Contract all tensors in a single row of the lattice.

        Parameters:
        row_tensors: List[Tensor] - A list of tensors in a row.

        Returns:
        Tensor: The contracted row as a single tensor.
        """
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            contracted, t = self.ensure_bond_match(contracted, t)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, t)
        return contracted

    def full_contraction(self):
        """
        Perform a full contraction of the PEPS tensor network to compute the final value.

        Returns:
        Tensor: The fully contracted scalar result.
        """
        row_contractions = []
        for row in self.tensors:
            row_contracted = self.contract_row(row)
            row_contractions.append(row_contracted)

        # Contract all rows together
        contracted = row_contractions[0]
        for row in row_contractions[1:]:
            contracted, row = self.ensure_bond_match(contracted, row)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, row)

        return contracted.sum()  # Reduce to scalar

# Define the PEPS model
size = (3, 3)  # PEPS lattice size (3x3 grid)
bond_dim = 3  # Bond dimension
peps = PEPS(size, bond_dim)

# Apply Hamiltonian
hamiltonian = peps.apply_hamiltonian()

# Compute observable
observable = peps.compute_observable()

# Print observable result
observable_result = observable.sum()
print(f"Observable result: {observable_result.item()}")

# Perform full contraction and print final result
full_contraction_result = peps.full_contraction()
print(f"Full contraction result (scalar): {full_contraction_result.item()}")

In [ ]:
import torch
import opt_einsum as oe

class PEPS:
    def __init__(self, size, bond_dim):
        """
        Initialize a PEPS model with a given lattice size and bond dimension.

        Parameters:
        size: Tuple[int, int] - The PEPS lattice dimensions (rows, columns).
        bond_dim: int - Bond dimension for the PEPS tensors.
        """
        self.size = size
        self.bond_dim = bond_dim
        self.tensors = self.initialize_tensors()

    def initialize_tensors(self):
        """
        Randomly initialize the PEPS tensors for each lattice site.

        Returns:
        List[List[Tensor]]: A 2D grid of tensors, each of shape (bond_dim, bond_dim, bond_dim, bond_dim).
        """
        tensors = []
        for i in range(self.size[0]):
            row = []
            for j in range(self.size[1]):
                tensor_shape = [self.bond_dim] * 4
                row.append(torch.rand(*tensor_shape, requires_grad=True))
            tensors.append(row)
        return tensors

    def apply_hamiltonian(self):
        """
        Apply a Hamiltonian to the PEPS lattice (dummy implementation with random tensor).

        Returns:
        Tensor: A tensor representing the Hamiltonian (random for demonstration).
        """
        h = torch.rand(self.size, requires_grad=True)
        print(f"Applying Hamiltonian: {h.shape}")
        return h

    def compute_observable(self):
        """
        Compute a dummy observable on the PEPS lattice (random tensor for demonstration).

        Returns:
        Tensor: A tensor representing the observable (random for demonstration).
        """
        obs = torch.rand(self.size, requires_grad=True)
        print(f"Computing observable: {obs.shape}")
        return obs

    def ensure_bond_match(self, t1, t2):
        """
        Check that the bond dimensions of two tensors match for contraction.

        Parameters:
        t1, t2: Tensor - Tensors to be contracted.

        Returns:
        Tuple[Tensor, Tensor]: The original tensors after validation.
        """
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")
        if t1.shape[-1] != t2.shape[0]:
            raise ValueError("Bond dimensions mismatch. Tensor contraction invalid.")
        return t1, t2

    def contract_row(self, row_tensors):
        """
        Contract all tensors in a single row of the lattice.

        Parameters:
        row_tensors: List[Tensor] - A list of tensors in a row.

        Returns:
        Tensor: The contracted row as a single tensor.
        """
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            contracted, t = self.ensure_bond_match(contracted, t)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, t)
        return contracted

    def full_contraction(self):
        """
        Perform a full contraction of the PEPS tensor network to compute the final value.

        Returns:
        Tensor: The fully contracted scalar result.
        """
        row_contractions = []
        for row in self.tensors:
            row_contracted = self.contract_row(row)
            row_contractions.append(row_contracted)

        # Contract all rows together
        contracted = row_contractions[0]
        for row in row_contractions[1:]:
            contracted, row = self.ensure_bond_match(contracted, row)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, row)

        return contracted.sum()  # Reduce to scalar

# Define the PEPS model
size = (3, 3)  # PEPS lattice size (3x3 grid)
bond_dim = 3  # Bond dimension
peps = PEPS(size, bond_dim)

# Apply Hamiltonian
hamiltonian = peps.apply_hamiltonian()

# Compute observable
observable = peps.compute_observable()

# Print observable result
observable_result = observable.sum()
print(f"Observable result: {observable_result.item()}")

# Perform full contraction and print final result
full_contraction_result = peps.full_contraction()
print(f"Full contraction result (scalar): {full_contraction_result.item()}")

In [ ]:
import torch
import opt_einsum as oe

class PEPS:
    def __init__(self, size, bond_dim):
        """
        Initialize a PEPS model with a given lattice size and bond dimension.

        Parameters:
        size: Tuple[int, int] - The PEPS lattice dimensions (rows, columns).
        bond_dim: int - Bond dimension for the PEPS tensors.
        """
        self.size = size
        self.bond_dim = bond_dim
        self.tensors = self.initialize_tensors()

    def initialize_tensors(self):
        """
        Randomly initialize the PEPS tensors for each lattice site.

        Returns:
        List[List[Tensor]]: A 2D grid of tensors, each of shape (bond_dim, bond_dim, bond_dim, bond_dim).
        """
        tensors = []
        for i in range(self.size[0]):
            row = []
            for j in range(self.size[1]):
                tensor_shape = [self.bond_dim] * 4
                row.append(torch.rand(*tensor_shape, requires_grad=True))
            tensors.append(row)
        return tensors

    def apply_hamiltonian(self):
        """
        Apply a Hamiltonian to the PEPS lattice (dummy implementation with random tensor).

        Returns:
        Tensor: A tensor representing the Hamiltonian (random for demonstration).
        """
        h = torch.rand(self.size, requires_grad=True)
        print(f"Applying Hamiltonian: {h.shape}")
        return h

    def compute_observable(self):
        """
        Compute a dummy observable on the PEPS lattice (random tensor for demonstration).

        Returns:
        Tensor: A tensor representing the observable (random for demonstration).
        """
        obs = torch.rand(self.size, requires_grad=True)
        print(f"Computing observable: {obs.shape}")
        return obs

    def ensure_bond_match(self, t1, t2):
        """
        Ensure that the bond dimensions of two tensors match for contraction.

        Parameters:
        t1, t2: Tensor - Tensors to be contracted.

        Returns:
        Tuple[Tensor, Tensor]: The original tensors after validation.
        """
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")
        if t1.shape[-1] != t2.shape[0]:
            raise ValueError("Bond dimensions mismatch. Tensor contraction invalid.")
        return t1, t2

    def contract_row(self, row_tensors):
        """
        Contract all tensors in a single row of the lattice.

        Parameters:
        row_tensors: List[Tensor] - A list of tensors in a row.

        Returns:
        Tensor: The contracted row as a single tensor.
        """
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            contracted, t = self.ensure_bond_match(contracted, t)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, t)
        return contracted

    def full_contraction(self):
        """
        Perform a full contraction of the PEPS tensor network to compute the final value.

        Returns:
        Tensor: The fully contracted scalar result.
        """
        row_contractions = []
        for row in self.tensors:
            row_contracted = self.contract_row(row)
            row_contractions.append(row_contracted)

        # Contract all rows together
        contracted = row_contractions[0]
        for row in row_contractions[1:]:
            contracted, row = self.ensure_bond_match(contracted, row)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, row)

        return contracted.sum()  # Reduce to scalar


# Define the PEPS model
size = (3, 3)  # PEPS lattice size (3x3 grid)
bond_dim = 3  # Bond dimension
peps = PEPS(size, bond_dim)

# Apply Hamiltonian
hamiltonian = peps.apply_hamiltonian()

# Compute observable
observable = peps.compute_observable()

# Print observable result
observable_result = observable.sum()
print(f"Observable result: {observable_result.item()}")

# Perform full contraction and print final result
full_contraction_result = peps.full_contraction()
print(f"Full contraction result (scalar): {full_contraction_result.item()}")

In [ ]:
import torch
import opt_einsum as oe


class PEPS:
    def __init__(self, size, bond_dim):
        """
        Initialize a PEPS model with a given lattice size and bond dimension.

        Parameters:
        size: Tuple[int, int] - The PEPS lattice dimensions (rows, columns).
        bond_dim: int - Bond dimension for the PEPS tensors.
        """
        self.size = size
        self.bond_dim = bond_dim
        self.tensors = self.initialize_tensors()

    def initialize_tensors(self):
        """
        Randomly initialize the PEPS tensors for each lattice site.

        Returns:
        List[List[Tensor]]: A 2D grid of tensors, each of shape (bond_dim, bond_dim, bond_dim, bond_dim).
        """
        tensors = []
        for i in range(self.size[0]):
            row = []
            for j in range(self.size[1]):
                # Initialize 4D tensors for each PEPS site
                tensor_shape = [self.bond_dim] * 4
                row.append(torch.rand(*tensor_shape, requires_grad=True))
            tensors.append(row)
        return tensors

    def apply_hamiltonian(self):
        """
        Apply a Hamiltonian to the PEPS lattice (random tensor for demonstration).

        Returns:
        Tensor: A tensor representing the Hamiltonian.
        """
        hamiltonian = torch.rand(self.size, requires_grad=True)
        print(f"Applying Hamiltonian: {hamiltonian.shape}")
        return hamiltonian

    def compute_observable(self):
        """
        Compute a dummy observable on the PEPS lattice.

        Returns:
        Tensor: A tensor representing the observable (random for demonstration).
        """
        observable = torch.rand(self.size, requires_grad=True)
        print(f"Computing observable: {observable.shape}")
        return observable

    def ensure_bond_match(self, t1, t2):
        """
        Ensure that the bond dimensions of two tensors match for contraction.

        Parameters:
        t1, t2: Tensor - Tensors to be contracted.

        Returns:
        Tuple[Tensor, Tensor]: The original tensors after validation.
        """
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")
        if t1.shape[-1] != t2.shape[0]:
            raise ValueError("Bond dimensions mismatch. Tensor contraction invalid.")
        return t1, t2

    def contract_row(self, row_tensors):
        """
        Contract all tensors in a single row of the lattice.

        Parameters:
        row_tensors: List[Tensor] - A list of tensors in a row.

        Returns:
        Tensor: The contracted row as a single tensor.
        """
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            contracted, t = self.ensure_bond_match(contracted, t)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, t)
        return contracted

    def full_contraction(self):
        """
        Perform a full contraction of the PEPS tensor network to compute the final value.

        Returns:
        Tensor: The fully contracted scalar result.
        """
        # Step 1: Contract each row into a single tensor
        row_contractions = []
        for row in self.tensors:
            row_contracted = self.contract_row(row)
            row_contractions.append(row_contracted)

        # Step 2: Contract all rows together into a single scalar
        contracted = row_contractions[0]
        for row in row_contractions[1:]:
            contracted, row = self.ensure_bond_match(contracted, row)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, row)

        return contracted.sum()  # Reduce to scalar


# Define the PEPS model
size = (3, 3)  # PEPS lattice size (3x3 grid)
bond_dim = 3  # Bond dimension
peps = PEPS(size, bond_dim)

# Step 1: Apply Hamiltonian
hamiltonian = peps.apply_hamiltonian()

# Step 2: Compute observable
observable = peps.compute_observable()

# Print observable result
observable_result = observable.sum()
print(f"Observable result: {observable_result.item()}")

# Step 3: Perform full contraction and print final result
full_contraction_result = peps.full_contraction()
print(f"Full contraction result (scalar): {full_contraction_result.item()}")

In [ ]:
import torch
import opt_einsum as oe

class PEPS:
    def __init__(self, size, bond_dim):
        """
        Initialize a PEPS model with a given lattice size and bond dimension.

        Parameters:
        size: Tuple[int, int] - The PEPS lattice dimensions (rows, columns).
        bond_dim: int - Bond dimension for the PEPS tensors.
        """
        self.size = size
        self.bond_dim = bond_dim
        self.tensors = self.initialize_tensors()

    def initialize_tensors(self):
        """
        Randomly initialize the PEPS tensors for each lattice site.

        Returns:
        List[List[Tensor]]: A 2D grid of tensors, each of shape (bond_dim, bond_dim, bond_dim, bond_dim).
        """
        tensors = []
        for i in range(self.size[0]):
            row = []
            for j in range(self.size[1]):
                tensor_shape = [self.bond_dim] * 4  # Bond dimensions: left, right, up, down
                row.append(torch.rand(*tensor_shape, requires_grad=True))
            tensors.append(row)
        return tensors

    def apply_hamiltonian(self):
        """
        Apply a random Hamiltonian to the PEPS lattice (for demonstration).

        Returns:
        Tensor: A tensor representing the Hamiltonian.
        """
        hamiltonian = torch.rand(self.size, requires_grad=True)
        print(f"Applying Hamiltonian: {hamiltonian.shape}")
        return hamiltonian

    def compute_observable(self):
        """
        Compute a dummy observable on the PEPS lattice.

        Returns:
        Tensor: A tensor representing the observable (random for demonstration).
        """
        observable = torch.rand(self.size, requires_grad=True)
        print(f"Computing observable: {observable.shape}")
        return observable

    def ensure_bond_match(self, t1, t2):
        """
        Ensure that the bond dimensions of two tensors match for contraction.

        Parameters:
        t1, t2: Tensor - Tensors to be contracted.

        Returns:
        Tuple[Tensor, Tensor]: The original tensors after validation.
        """
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")
        if t1.shape[-1] != t2.shape[0]:
            raise ValueError("Bond dimensions mismatch. Tensor contraction invalid.")
        return t1, t2

    def contract_row(self, row_tensors):
        """
        Contract all tensors in a single row of the lattice.

        Parameters:
        row_tensors: List[Tensor] - A list of tensors in a row.

        Returns:
        Tensor: The contracted row as a single tensor.
        """
        contracted = row_tensors[0]
        for t in row_tensors[1:]:
            contracted, t = self.ensure_bond_match(contracted, t)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, t)
        return contracted

    def contract_full_lattice(self):
        """
        Perform a full contraction of the PEPS lattice, reducing it to a scalar.

        Returns:
        Tensor: The final scalar result after full contraction.
        """
        # Step 1: Contract each row
        row_contractions = []
        for row in self.tensors:
            row_contracted = self.contract_row(row)
            row_contractions.append(row_contracted)

        # Step 2: Contract all rows
        contracted = row_contractions[0]
        for row in row_contractions[1:]:
            contracted, row = self.ensure_bond_match(contracted, row)
            contraction_string = "abcd,bcde->acde"
            contracted = oe.contract(contraction_string, contracted, row)

        return contracted.sum()  # Reduce to scalar

    def compute(self):
        """
        Execute the full PEPS computation: Hamiltonian, observable, and full contraction.
        """
        # Step 1: Apply Hamiltonian
        hamiltonian = self.apply_hamiltonian()

        # Step 2: Compute observable
        observable = self.compute_observable()
        observable_result = observable.sum().item()
        print(f"Observable result: {observable_result}")

        # Step 3: Perform full contraction
        full_contraction_result = self.contract_full_lattice().item()
        print(f"Full contraction result (scalar): {full_contraction_result}")


# Define PEPS parameters
size = (3, 3)  # Lattice size
bond_dim = 3  # Bond dimension

# Initialize and compute with PEPS
peps = PEPS(size, bond_dim)
peps.compute()

In [ ]:
import torch
import opt_einsum as oe

class PEPS:
    def __init__(self, size, bond_dim):
        self.size = size
        self.bond_dim = bond_dim
        self.tensors = [[torch.rand(bond_dim, bond_dim, bond_dim, bond_dim, requires_grad=True)
                         for _ in range(size[1])] for _ in range(size[0])]

    def apply_hamiltonian(self):
        """Apply a simple Hamiltonian (as a placeholder)."""
        hamiltonian = torch.rand(self.size)
        print(f"Applying Hamiltonian: {hamiltonian.shape}")
        return hamiltonian

    def compute_observable(self):
        """Compute a simple observable (as a placeholder)."""
        observable = torch.rand(self.size)
        print(f"Computing observable: {observable.shape}")
        return observable.sum()

    def ensure_bond_match(self, t1, t2):
        """Ensure tensors have matching bonds for contraction."""
        if t1.shape != t2.shape:
            raise ValueError(f"Bond mismatch: t1.shape={t1.shape}, t2.shape={t2.shape}")
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")

    def contract_two_tensors(self, t1, t2):
        """Contract two tensors."""
        self.ensure_bond_match(t1, t2)
        return torch.tensordot(t1, t2, dims=([3, 2], [1, 0]))  # Adjust the contraction indices

    def contract_row(self, row):
        """Contract a row of tensors."""
        result = row[0]
        for tensor in row[1:]:
            result = self.contract_two_tensors(result, tensor)
        return result

    def contract_full_lattice(self):
        """Contract the entire PEPS lattice."""
        contracted_rows = [self.contract_row(row) for row in self.tensors]
        result = contracted_rows[0]
        for row in contracted_rows[1:]:
            result = self.contract_two_tensors(result, row)
        print(f"Full contraction result (scalar): {result.sum().item()}")
        return result.sum()

def optimize_peps(peps, lr=1e-2, steps=100):
    """Optimize PEPS tensors to minimize energy."""
    optimizer = torch.optim.Adam([tensor for row in peps.tensors for tensor in row], lr=lr)
    for step in range(steps):
        optimizer.zero_grad()
        energy = peps.contract_full_lattice()  # Use full contraction as energy
        loss = energy
        loss.backward()
        optimizer.step()
        if step % 10 == 0:
            print(f"Step {step}, Energy: {loss.item():.4f}")

# Example usage
if __name__ == "__main__":
    size = (3, 3)  # PEPS grid size
    bond_dim = 3   # Bond dimension
    peps = PEPS(size, bond_dim)

    # Step 1: Apply Hamiltonian
    hamiltonian = peps.apply_hamiltonian()

    # Step 2: Compute observable
    observable = peps.compute_observable()
    print(f"Observable result: {observable}")

    # Step 3: Optimize PEPS tensors
    print("Starting optimization...")
    optimize_peps(peps, lr=1e-2, steps=50)

In [ ]:
import torch
import opt_einsum as oe

class PEPS:
    def __init__(self, size, bond_dim):
        self.size = size
        self.bond_dim = bond_dim
        self.tensors = [[torch.rand(bond_dim, bond_dim, bond_dim, bond_dim, requires_grad=True)
                         for _ in range(size[1])] for _ in range(size[0])]

    def apply_hamiltonian(self):
        """Apply a simple Hamiltonian (as a placeholder)."""
        hamiltonian = torch.rand(self.size)
        print(f"Applying Hamiltonian: {hamiltonian.shape}")
        return hamiltonian

    def compute_observable(self):
        """Compute a simple observable (as a placeholder)."""
        observable = torch.rand(self.size)
        print(f"Computing observable: {observable.shape}")
        return observable.sum()

    def ensure_bond_match(self, t1, t2):
        """Ensure tensors have matching bonds for contraction."""
        if t1.shape != t2.shape:
            raise ValueError(f"Bond mismatch: t1.shape={t1.shape}, t2.shape={t2.shape}")
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")

    def contract_two_tensors(self, t1, t2):
        """Contract two tensors."""
        self.ensure_bond_match(t1, t2)
        return torch.tensordot(t1, t2, dims=([3, 2], [1, 0]))  # Adjust the contraction indices

    def contract_row(self, row):
        """Contract a row of tensors."""
        result = row[0]
        for tensor in row[1:]:
            result = self.contract_two_tensors(result, tensor)
        return result

    def contract_full_lattice(self):
        """Contract the entire PEPS lattice."""
        contracted_rows = [self.contract_row(row) for row in self.tensors]
        result = contracted_rows[0]
        for row in contracted_rows[1:]:
            result = self.contract_two_tensors(result, row)
        print(f"Full contraction result (scalar): {result.sum().item()}")
        return result.sum()

def optimize_peps(peps, lr=1e-2, steps=100):
    """Optimize PEPS tensors to minimize energy."""
    optimizer = torch.optim.Adam([tensor for row in peps.tensors for tensor in row], lr=lr)
    for step in range(steps):
        optimizer.zero_grad()
        energy = peps.contract_full_lattice()  # Use full contraction as energy
        loss = energy
        loss.backward()
        optimizer.step()
        if step % 10 == 0:
            print(f"Step {step}, Energy: {loss.item():.4f}")

# Example usage
if __name__ == "__main__":
    size = (3, 3)  # PEPS grid size
    bond_dim = 3   # Bond dimension
    peps = PEPS(size, bond_dim)

    # Step 1: Apply Hamiltonian
    hamiltonian = peps.apply_hamiltonian()

    # Step 2: Compute observable
    observable = peps.compute_observable()
    print(f"Observable result: {observable}")

    # Step 3: Optimize PEPS tensors
    print("Starting optimization...")
    optimize_peps(peps, lr=1e-2, steps=50)

In [ ]:
import torch
import opt_einsum as oe

class PEPS:
    def __init__(self, size, bond_dim, device='cuda'):
        self.size = size
        self.bond_dim = bond_dim
        self.device = device
        self.tensors = [[torch.rand(bond_dim, bond_dim, bond_dim, bond_dim, requires_grad=True, device=device)
                         for _ in range(size[1])] for _ in range(size[0])]

    def apply_hamiltonian(self):
        hamiltonian = torch.rand(self.size, device=self.device)
        print(f"Applying Hamiltonian: {hamiltonian.shape}")
        return hamiltonian

    def compute_observable(self):
        observable = torch.rand(self.size, device=self.device)
        print(f"Computing observable: {observable.shape}")
        return observable.sum()

    def ensure_bond_match(self, t1, t2):
        if t1.shape != t2.shape:
            raise ValueError(f"Bond mismatch: t1.shape={t1.shape}, t2.shape={t2.shape}")
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")

    def contract_two_tensors(self, t1, t2):
        self.ensure_bond_match(t1, t2)
        return torch.tensordot(t1, t2, dims=([3, 2], [1, 0]))

    def contract_row(self, row):
        result = row[0]
        for tensor in row[1:]:
            result = self.contract_two_tensors(result, tensor)
        return result

    def contract_full_lattice(self):
        contracted_rows = [self.contract_row(row) for row in self.tensors]
        result = contracted_rows[0]
        for row in contracted_rows[1:]:
            result = self.contract_two_tensors(result, row)
        print(f"Full contraction result (scalar): {result.sum().item()}")
        return result.sum()

    # Extend observable computation to include multi-site observables
    def compute_multi_site_observable(self):
        observables = []
        for i in range(self.size[0] - 1):
            for j in range(self.size[1] - 1):
                observable = torch.tensordot(self.tensors[i][j], self.tensors[i + 1][j + 1], dims=([2, 3], [0, 1]))
                observables.append(observable)
        total_observable = sum(observables)
        print(f"Multi-site observable: {total_observable.item()}")
        return total_observable

def optimize_peps(peps, lr=1e-2, steps=100):
    optimizer = torch.optim.Adam([tensor for row in peps.tensors for tensor in row], lr=lr)
    for step in range(steps):
        optimizer.zero_grad()
        energy = peps.contract_full_lattice()
        loss = energy
        loss.backward()
        optimizer.step()
        if step % 10 == 0:
            print(f"Step {step}, Energy: {loss.item():.4f}")

# Example usage
if __name__ == "__main__":
    size = (3, 3)
    bond_dim = 3
    peps = PEPS(size, bond_dim)

    # Step 1: Apply Hamiltonian
    hamiltonian = peps.apply_hamiltonian()

    # Step 2: Compute observable
    observable = peps.compute_observable()
    print(f"Observable result: {observable}")

    # Step 3: Compute multi-site observable
    multi_site_observable = peps.compute_multi_site_observable()
    print(f"Multi-site observable result: {multi_site_observable}")

    # Step 4: Optimize PEPS tensors
    print("Starting optimization...")
    optimize_peps(peps, lr=1e-2, steps=50)